In [1]:
import os
import json
from dotenv import load_dotenv
from swarm import Swarm, Agent
import openai
from prompts import *

from Chat import AgentClient
from Toolbox import ImageProcessingToolBoxes

# Set the environment variable for the OpenAI API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

In [2]:
test_image_path = './test-images/a0001-jmac_DSC1459.dng'
output_dir_path = f'./test-output/{os.path.splitext(os.path.basename(test_image_path))[0]}'
image_process = ImageProcessingToolBoxes(image_path=test_image_path, output_dir_name=output_dir_path, debug=True)
chat_client = AgentClient(api_key=api_key, toolbox_instance=image_process, debug=False)

In [3]:
chat_client.agent_interaction(system_prompt=image_content_analyzer_prompt())
chat_client.agent_interaction(system_prompt=global_retouching_concept_prompt(function_aspects=image_process.get_function_short_description()))
chat_client.agent_get_plan(system_prompt=retouching_planing_system_prompt(), user_prompt=retouching_planing_user_prompt(image_process.get_function_names()))

while image_process.processing_plan:
    current_function = image_process.processing_plan[0]

    if image_process.plan_status != current_function:  # 如果正在执行的function和现在的一样，说明是重新执行的。否则是重新开始
        image_process.set_plan_status(current_function)
        image_process.set_satisfactory_status(False)  # 开始新的function默认不满意

        chat_client.agent_execute_plan(
            system_prompt=retouching_execute_system_prompt(),
            user_prompt=retouching_execute_user_prompt(current_function),
            current_function_name=current_function)
    else:
        chat_client.agent_execute_plan(
            system_prompt=retouching_execute_system_prompt(),
            user_prompt=retouching_re_execute_user_prompt(current_function),
            current_function_name=current_function)

    chat_client.agent_reflection_plan(
        system_prompt=retouching_reflection_system_prompt())

    if image_process.satisfactory_status:
        image_process.finish_current_plan()



`func_to_return_responses`, response: 

**Analysis of the Main Content:**
The image depicts a classic desert landscape featuring a prominent red sandstone butte, likely a part of Monument Valley. The butte stands imposingly against a backdrop of a partly cloudy sky, emphasizing its height and rugged texture. Shrubs and small vegetation dot the arid, reddish-brown terrain in the foreground, adding a sense of scale and depth to the scene. 

**Evaluation of Photographic Value:**
The photograph is captured from a low vantage point, highlighting the grandeur of the butte and making it the focal point of the composition. The natural lighting casts shadows that add contrast and bring out the textures of the butte and the desert floor. The composition follows a central alignment, drawing the viewer's eye directly to the butte while maintaining balance with the sky and landscape.

**Speculation on Emotions and Artistic Values:**
The image conveys a sense of solitude and timelessness, often as

In [4]:
image_process.check_history_messages()


Message 1 | Role: assistant
--------------------------------------------------------------------------------
**Photo Content Analysis:**  
The image features a person standing near a rocky shoreline, gazing towards the ocean. The individual is wearing casual clothing, including a striped top and loose pants. The scene is set during daylight with visible clouds and a clear blue sky.

**Photographic Value Evaluation:**  
The photo uses a side perspective to capture the subject, with the ocean and rocks providing a scenic backdrop. The lighting appears natural, with shadows indicating the sun's direction is slightly behind. The composition draws attention to the solitary figure against the vastness of the sea, emphasizing the contrast between the individual's contemplation and nature's expanse.

**Emotional and Artistic Speculation:**  
The image evokes feelings of solitude, introspection, and peace, with the person looking toward the horizon. The artistic message might explore themes of